<a href="https://colab.research.google.com/github/DebbabiSkander/BrainOS/blob/main/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets evaluate accelerate arabic-reshaper farasapy



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
import re
import torch
from datasets import Dataset

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load Tunisian Dialect Corpus
dataset = load_dataset("arbml/Tunisian_Dialect_Corpus")

# Explore structure
print(dataset)
print(dataset['train'][0])

# Convert to pandas for inspection
df = pd.DataFrame(dataset['train'])
print(df.head())

# Check label distribution
print(df['label'].value_counts())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


data/train-00000-of-00001.parquet:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49889 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'label'],
        num_rows: 49889
    })
})
{'Tweet': 'Nn mouch 7louwa faza', 'label': 1}
                                               Tweet  label
0                               Nn mouch 7louwa faza      1
1                            mabladkom 3bed tfouuuuh      1
2  تفووووووه عليك و علا والديك على عايلتك و على ا...      1
3                              لا يليق بهذا البرنامج      1
4                                              رهدان      1
label
0    36785
1    13104
Name: count, dtype: int64


In [ ]:
import re
import arabic_reshaper
from farasa.pos import FarasaPOSTagger

# Optional: only if you plan to reshape Arabic later; not needed for TunBERT tokenization
def clean_tunisian_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[ـ]+', '', text)                     # Tatweel
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)  # Diacritics
    text = re.sub(r'[^\w\s\u0600-\u06FF0-9]+', ' ', text)     # Keep Arabic letters, digits, Latin
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
print(clean_tunisian_text("تفووووووه عليك و علا والديك!!!"))
print(clean_tunisian_text("mouch 7louwa faza 😒"))


تفووووووه عليك و علا والديك
mouch 7louwa faza


In [ ]:
def preprocess_function(examples):
    examples["Tweet"] = [clean_tunisian_text(t) for t in examples["Tweet"]]
    return examples

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/49889 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tunis-ai/TunBERT")

def tokenize_function(examples):
    return tokenizer(
        examples["Tweet"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Map:   0%|          | 0/49889 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset["train"][0])


{'Tweet': 'Nn mouch 7louwa faza', 'labels': tensor(1), 'input_ids': tensor([  101,  1050,  2078,  9587, 10875,  1021, 23743,  4213,  6904,  4143,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,    

## step 3

In [ ]:
from datasets import DatasetDict

# 80% train / 20% validation split
split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.2, seed=42)

# Combine into DatasetDict for Trainer
dataset_dict = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

print(dataset_dict)


DatasetDict({
    train: Dataset({
        features: ['Tweet', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 39911
    })
    validation: Dataset({
        features: ['Tweet', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9978
    })
})


In [ ]:
import numpy as np

def label_distribution(ds):
    labels = ds["labels"]
    unique, counts = np.unique(labels, return_counts=True)
    return dict(zip(unique, counts))

print("Train distribution:", label_distribution(dataset_dict["train"]))
print("Validation distribution:", label_distribution(dataset_dict["validation"]))


Train distribution: {np.int64(0): np.int64(29413), np.int64(1): np.int64(10498)}
Validation distribution: {np.int64(0): np.int64(7372), np.int64(1): np.int64(2606)}


In [ ]:
import torch

train_labels = torch.tensor(dataset_dict["train"]["labels"])
num_samples = len(train_labels)
class_counts = torch.bincount(train_labels)
class_weights = num_samples / (2.0 * class_counts)
print("Class Weights:", class_weights)


Class Weights: tensor([0.6785, 1.9009])


In [ ]:
from transformers import AutoModelForSequenceClassification

MODEL_ID = "tunis-ai/TunBERT"

# Set the explicit mapping based on your dataset documentation: 0=Positive, 1=Negative
id2label_map = {
    0: "Positive",  # 73.7%
    1: "Negative"   # 26.3%
}
label2id_map = {
    "Positive": 0,
    "Negative": 1
}

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    id2label=id2label_map,
    label2id=label2id_map
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tunis-ai/TunBERT and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.output.Laye

In [ ]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=predictions, references=labels, average="binary")["f1"]
    }


In [ ]:
from transformers import Trainer
import torch.nn as nn

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=0):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
import transformers
print(transformers.__version__)

4.57.1


In [ ]:
print(TrainingArguments)


<class 'transformers.training_args.TrainingArguments'>


In [ ]:
training_args = TrainingArguments(
    output_dir="./tunbert-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    warmup_steps=500,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=256,
    num_train_epochs=8,
    weight_decay=0.005,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    fp16=True,
)

In [ ]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-228779528.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.456500,0.451129,0.783724,0.563818,0.759401,0.647155
2,0.417200,0.446676,0.715975,0.476914,0.903684,0.624337
3,0.387700,0.439154,0.807376,0.604844,0.757099,0.672461
4,0.372700,0.449935,0.818200,0.637024,0.706447,0.669942
5,0.337100,0.455244,0.828122,0.665061,0.688795,0.676720
6,0.324400,0.434345,0.785027,0.557582,0.856485,0.675443
7,0.301000,0.440248,0.819603,0.621826,0.789332,0.695637
8,0.320400,0.437630,0.800862,0.583446,0.830391,0.685352


TrainOutput(global_step=4992, training_loss=0.3589959415869835, metrics={'train_runtime': 1875.0498, 'train_samples_per_second': 170.282, 'train_steps_per_second': 2.662, 'total_flos': 2.100205066094592e+16, 'train_loss': 0.3589959415869835, 'epoch': 8.0})

In [ ]:
print("Training dataset size:", len(dataset_dict["train"]))
print("Validation dataset size:", len(dataset_dict["validation"]))


Training dataset size: 39911
Validation dataset size: 9978


In [ ]:
import torch

def predict_sentiment(text):
    # Clean text (use the same cleaning you used during training)
    def clean_tunisian_text(text):
        import re
        text = re.sub(r'[ـ]+', '', text)  # Remove Tatweel
        text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)  # Remove diacritics
        text = re.sub(r'[^\w\s\u0600-\u06FF0-9]+', ' ', text)  # Keep Arabic, Latin letters, numbers
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    cleaned_text = clean_tunisian_text(text)

    # Tokenize input
    inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Move tensors to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        confidence, predicted_class = torch.max(probabilities, dim=1)

    # Map predicted label to string
    label_map = {0: "Positive", 1: "Negative"}
    return label_map[predicted_class.item()], confidence.item()


In [ ]:
test_texts = [
    "أنا فرحان برشا اليوم",
    "برشة حاجات ما عجبونيش",
    "الجو اليوم باهي",
    "ما نحبش الخدمة هاذي"
]

for text in test_texts:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}, Confidence: {confidence:.2f}")
    print("-" * 40)


Text: أنا فرحان برشا اليوم
Sentiment: Positive, Confidence: 0.80
----------------------------------------
Text: برشة حاجات ما عجبونيش
Sentiment: Negative, Confidence: 0.75
----------------------------------------
Text: الجو اليوم باهي
Sentiment: Positive, Confidence: 0.86
----------------------------------------
Text: ما نحبش الخدمة هاذي
Sentiment: Positive, Confidence: 0.78
----------------------------------------


In [ ]:
test_texts_advanced = [
    "ربي يقدر الخير، أما ما نتصورش.",
    "تي عيش خوك، كرهتني في حياتي!",
    "يا سلام، عظمة موش عادية!",
    "هبطتلي مورالي، الله يهديك.",
    "الخدمة كملت على أحسن ما يرام.",
    "فاتت ساعة ومانيش فاهم شي.",
    "برا عاد، خويا العزيز!",
    "نحب نجي نعاونك في وقته."
]

print("--- Running Advanced Stress Tests (Tunisian Dialect) ---")
for text in test_texts_advanced:
    # Assuming 'predict_sentiment' is defined and the label map is corrected
    sentiment, confidence = predict_sentiment(text)
    print(f": {text}")
    print(f": {sentiment}, : {confidence:.2f}")
    print("-" * 40)

--- Running Advanced Stress Tests (Tunisian Dialect) ---
: ربي يقدر الخير، أما ما نتصورش.
: Positive, : 0.96
----------------------------------------
: تي عيش خوك، كرهتني في حياتي!
: Negative, : 0.53
----------------------------------------
: يا سلام، عظمة موش عادية!
: Negative, : 0.94
----------------------------------------
: هبطتلي مورالي، الله يهديك.
: Positive, : 0.80
----------------------------------------
: الخدمة كملت على أحسن ما يرام.
: Positive, : 0.56
----------------------------------------
: فاتت ساعة ومانيش فاهم شي.
: Negative, : 0.60
----------------------------------------
: برا عاد، خويا العزيز!
: Negative, : 0.85
----------------------------------------
: نحب نجي نعاونك في وقته.
: Positive, : 0.66
----------------------------------------


In [ ]:
test_sentences = [
    # 🟢 Positive
    "أنا فرحان برشا اليوم",
    "الجو اليوم باهي ياسر",
    "حاجة تفرح بصراحة",
    "film hedha 3جبني برشا",

    # 🔴 Negative
    "ما نحبش الخدمة هاذي",
    "برشة حاجات ما عجبونيش",
    "البرنامج هذا خايب برشا",
    "tfou 3lik wallah",
]

for text in test_sentences:
    sentiment, confidence = predict_sentiment(text)
    print(f"Text: {text}")
    print(f"→ Predicted: {sentiment} (Confidence: {confidence:.2f})")
    print("-" * 60)



Text: أنا فرحان برشا اليوم
→ Predicted: Positive (Confidence: 0.80)
------------------------------------------------------------
Text: الجو اليوم باهي ياسر
→ Predicted: Positive (Confidence: 0.94)
------------------------------------------------------------
Text: حاجة تفرح بصراحة
→ Predicted: Positive (Confidence: 0.74)
------------------------------------------------------------
Text: film hedha 3جبني برشا
→ Predicted: Positive (Confidence: 0.72)
------------------------------------------------------------
Text: ما نحبش الخدمة هاذي
→ Predicted: Positive (Confidence: 0.78)
------------------------------------------------------------
Text: برشة حاجات ما عجبونيش
→ Predicted: Negative (Confidence: 0.75)
------------------------------------------------------------
Text: البرنامج هذا خايب برشا
→ Predicted: Negative (Confidence: 0.55)
------------------------------------------------------------
Text: tfou 3lik wallah
→ Predicted: Negative (Confidence: 1.00)
----------------------------------